In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
import time

In [80]:
# 获取需要访问的公司
firm_names_50_path = r'C:\Users\Lenovo\Desktop\爬虫课程作业\firm_names_50.csv'
firm_names_50_ls = list(pd.read_csv(firm_names_50_path, header=None)[0])

In [81]:
# 访问网页
driver = webdriver.Chrome()
driver.get('https://www.tianyancha.com/')

In [82]:
def find_information(name):    # name为公司名字
    # 点击搜索框，输入公司名字，按下回车键
    driver.find_element(By.XPATH, '//*[@id="page-container"]/div[1]/div/div[3]/div[2]/div[1]/div/input').send_keys(name, Keys.RETURN)
    time.sleep(3)

    # 点击第一个框中的公司
    driver.find_elements(By.CSS_SELECTOR, 'div.index_name__qEdWi')[0].click()
    time.sleep(3)
    
    # 切换到新的窗口
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(1)

    # 返回注册资本
    try:
        Registered_capital = driver.find_elements(By.XPATH, '//*[@id="page-root"]/div[3]/div[1]/div[3]/div/div[2]/div[2]/div[2]/div[1]/div/div[2]/table/tbody/tr[3]/td[2]/div')[0].text
    except:
        Registered_capital = 'NA'
    
    # 找到股东信息
    try:
        Shareholder_ls = []
        for element in driver.find_elements(By.CSS_SELECTOR, 'a.index_lazy-img-toco__EU_FE.link-click'):
            Shareholder_ls.append(element.text)
        if len(Shareholder_ls) > 10:
            Shareholder_ls = Shareholder_ls[:10]
    except:
        Shareholder_ls = []

    # 关闭当前的页面
    driver.close()
    time.sleep(1)

    # 切换到新的窗口
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(3)

    # 返回首页
    driver.find_element(By.XPATH, '//*[@id="page-header"]/div/a/i[2]').click()
    time.sleep(3)
    
    return Registered_capital,Shareholder_ls

# 得到中国TOP50企业的注册资本和前十大股东
firm_dic = {}
for firm_name in firm_names_50_ls:
    firm_dic[firm_name] = list(find_information(firm_name))

firm_dic

{'国家电网有限公司': ['82950000万人民币', ['国务院国有资产监督管理委员会']],
 '中国石油天然气集团有限公司': ['48690000万人民币', ['国务院国有资产监督管理委员会']],
 '中国石油化工集团有限公司': ['32654722.2万人民币', ['国务院国有资产监督管理委员会']],
 '中国建筑股份有限公司': ['3000000万人民币',
  ['中国建筑集团有限公司', '香港中央结算有限公司', '中国证券金融股份有限公司', '中央汇金资产管理有限责任公司']],
 '中国工商银行股份有限公司': ['35640625.7089万人民币',
  ['中央汇金投资有限责任公司',
   '中华人民共和国财政部',
   '香港中央结算（代理人）有限公司',
   '全国社会保障基金理事会',
   '中国证券金融股份有限公司',
   '香港中央结算有限公司',
   '中央汇金资产管理有限责任公司']],
 '中国建设银行股份有限公司': ['25001097.7486万人民币', ['中央汇金投资有限责任公司', '香港中央结算（代理人）有限公司']],
 '中国农业银行股份有限公司': ['34998303.3873万人民币',
  ['中央汇金投资有限责任公司', '中华人民共和国财政部', '香港中央结算（代理人）有限公司', '全国社会保障基金理事会']],
 '中国平安保险(集团)股份有限公司': ['1821023.4607万人民币',
  ['香港中央结算（代理人）有限公司',
   '深圳市投资控股有限公司',
   '香港中央结算有限公司',
   '中国证券金融股份有限公司',
   '中央汇金资产管理有限责任公司',
   '深业集团有限公司']],
 '中国中化控股有限责任公司': ['5525800万人民币', ['中华人民共和国国务院']],
 '中国铁路工程集团有限公司': ['1210000万人民币', ['国务院国有资产监督管理委员会']],
 '中国海洋石油集团有限公司': ['11380000万人民币', ['国务院国有资产监督管理委员会']],
 '中国铁道建筑集团有限公司': ['900000万人民币', ['国务院国有资产监督管理委员会']],
 '中国宝武钢铁集团有

In [83]:
# 输出结果
firm_result = pd.DataFrame(firm_dic , index = ['注册资本','前十大股东'])
# 行列转换
firm_result = firm_result.transpose()

# 返回一个csv文件
result_path = r'C:\Users\Lenovo\Desktop\爬虫课程作业\firm_names_50_result.csv'
firm_result.to_csv(result_path,encoding='utf-8-sig')

# 关闭WebDriver  
driver.quit()